In [ ]:
import os
import json
import glob
import torch
import numpy as np
from pathlib import Path
from natsort import natsorted
from pycocotools.coco import COCO
import matplotlib.patches as patches
from scipy.optimize import linear_sum_assignment

import matplotlib.pyplot as plt
%matplotlib inline

from endo_sim_data import EndoSimData
endo_data = EndoSimData()

%load_ext autoreload
%autoreload 2
################################################
# 1. Load raw data in the correct format
# 2. Pre-process and prepare data for evaluation
# 3. Metric computation
#################################################

In [ ]:
classes_to_eval = ['Nadelhalter', 'Nervhaken', 'Klappenschere', 'Atraum. Pinzette', 'Knotenschieber']
valid_classes = ['nadelhalter', 'nervhaken', 'klappenschere', 'atraum. pinzette', 'knotenschieber']
class_list = [cls.lower() if cls.lower() in valid_classes else None for cls in classes_to_eval]

In [ ]:
def _assign_ids(classes_list):
    ids = []
    for classes_t in classes_list:
        unique_classes_t = np.unique(classes_t)
        ids_t = [None] * len(classes_t)
        for class_ in unique_classes_t:
            id_ctr = 1
            for i, det in enumerate(classes_t):
                if det == class_:
                    id_ = (det * 100) + id_ctr
                    ids_t[i] = id_
                    id_ctr += 1

        ids.append(np.asarray(ids_t))
    return ids


def _load_raw_file(json_labels):
    num_timesteps = len(json_labels)
    data_keys = ['gt_ids', 'gt_classes', 'gt_dets']
    raw_data = {key: [None] * num_timesteps for key in data_keys}

    raw_data['gt_classes'] = [elem['labels'].cpu().detach().numpy() for elem in json_labels] # convert each labels into a np array
    raw_data['gt_ids'] = _assign_ids(raw_data['gt_classes'])
    raw_data['gt_dets'] = [np.asarray(elem['boxes']) for elem in json_labels]
    raw_data['num_timesteps'] = num_timesteps
    return raw_data


raw_gt_data = _load_raw_file(json_labels=json_gt_labels) # Give the GT labels read from json here
raw_tracker_data = _load_raw_file(json_labels=json_pred_labels) # Give the predicted labels read from the json here
raw_data = {**raw_tracker_data, **raw_gt_data}

preproc_seq_data = endo_data.get_preprocessed_seq_data(raw_data=raw_data,
                                                      cls='nadelhalter')

In [ ]:
from hota import HOTA

# Here you can also use the method in HOTA to average it over multiple classes
# or compute it for each calss and then average it
hota_metrics = HOTA()
metrics = hota_metrics.eval_sequence(preproc_seq_data)
print(metrics['HOTA'])